In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from gsheet import *
from mailer import *
from qr_code import *
from payment_data import *
import logging

Succesfully read config from: ['./config_prispevky.cfg']


In [3]:
logging.basicConfig(
    format="[%(levelname)s] %(message)s",
    level=logging.INFO
)

In [4]:
sheet = Sheets(SPREADSHEET_ID)

[INFO] file_cache is only supported with oauth2client<4.0.0


In [8]:
# sheet_names = sheet.get_sheet_names()
# sheet_name = '4pvs_all'
# sheet_name = 'PLATBY_PODZIM_2022_filtered'
sheet_name = 'TESTv2'
if 'TEST' in sheet_name:
    testmode=True
else:
    testmode=False
df = sheet.get_dataframe(sheet_name)

In [9]:
df

,Osoba,Evidenční číslo,Jednotka,Typ jednotky,Typ členství,Kategorie,Registrační číslo,E-mail (další),E-mail (hlavní),Mobil (další),...,Ostatní: mail,Ostatní: telefon,K platbe Jaro 2023,STS (Registrace),2/2021,Celkem Poplatek 2022,Poplatek 2/2022,STS,Zaplaceno,Zaplaceno STS
0,"Unpaid prispevky, NO sts",113.04.004-1,Albatrosové,Oddíl,Řádné,Skaut,10049,,mrkosjan@seznam.cz,,...,,,700,0,,1400,700,0,0,None
1,"Unpaid prispevky, paid sts",113.04.004-1,Kačky,Oddíl,Řádné,Skaut,20148,,,,...,,,500,400,,500,500,400,0,400
2,"Paid prispevky, unpaid sts",113.04.004-1,Bobři,Oddíl,Řádné,Skaut,14811,,,,...,,,700,200,,1400,700,200,700,None
3,"Unapid prispecky, unpaid STS",113.04.004-1,Albatrosové,Oddíl,Řádné,Skaut,10126,,,,...,,,700,200,700,1400,700,200,0,None
4,All paid - should not send,,Albatrosové,,,,12345,mrkosja1@fel.cvut.cz,,,...,,,700,200,700,1400,700,200,700,200
5,No payment - should not send,113.04.004-1,Rackové,Oddíl,Řádné,Skaut,10038,,,,...,,,0,0,,0,0,0,0,None
6,Invalid email - should not send,,Albatrosové,,,,476456,,,,...,dsadssadasga.cz,,0,0,,0,0,0,0,0
7,No email - should not send,113.04.004-1,Albatrosové,Oddíl,Řádné,Skaut,10039,,,,...,None,None,None,None,None,None,None,None,None,None
8,Socháň Gabriel,,Rackové,,,,20227,,hondzik@skaut.cz,,...,,,700,,700,700,,,0,None


In [10]:
NEEDED_COLS = NeededColumns()
pdf=PaymentsDataFrame(df, NEEDED_COLS)

[WARNING] Incorrect email address read: 'Ostatní: mail': 'dsadssadasga.cz'; addr-spec local part with no domain
/mnt/c/Users/mrkos/scth/projects/4pvs_prispevky_qr_kod/payment_data.py:114: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_fixed.loc[:, col] = pd.to_numeric(payments.loc[:, col])
[INFO] Of 9 people, missing email: 2,
	emailable paid: 3
	emailable unpaid: 4.
 Also there were 1 invalid emails.


In [11]:
pdf.df_missing_email

,Osoba,Jednotka,Registrační číslo,E-mail (hlavní),Otec: mail,Matka: mail,E-mail (další),Ostatní: mail,K platbe Jaro 2023,Zaplaceno,valid_addresses,invalid_addresses
6,Invalid email - should not send,Albatrosové,476456,None,None,None,None,dsadssadasga.cz,0,0,(),"((Ostatní: mail, dsadssadasga.cz),)"
7,No email - should not send,Albatrosové,10039,None,None,None,None,None,None,None,(),()


In [12]:
emails = []

"""
!!!!!WARNING! CURRENTLY SET TO EMAIL ALL AVAILABLE EMAIL ADDRESSES!!!!!

Modify this setting on second line:
    recepient_addresses = row.valid_addresses
    
Modify orderin in class NeededColumns
"""

for i, row in pdf.df_emailable_unpaid.iterrows():
    recepient_addresses = row.valid_addresses[:2]
    pi = PaymentInfo.from_df_row(row, NEEDED_COLS)
    email = PaymentEmail(pi, recepient_addresses, testmode=testmode)
    emails.append(email.msg)

In [13]:
pdf.df_emailable_unpaid

,Osoba,Jednotka,Registrační číslo,E-mail (hlavní),Otec: mail,Matka: mail,E-mail (další),Ostatní: mail,K platbe Jaro 2023,Zaplaceno,valid_addresses,invalid_addresses
0,"Unpaid prispevky, NO sts",Albatrosové,10049,mrkosjan@seznam.cz,mrkosjan@seznam.cz,mrkosjan@gmail.com,None,None,700,0,"(""E-mail (hlavní)"" <mrkosjan@seznam.cz>, ""Matk...",()
1,"Unpaid prispevky, paid sts",Kačky,20148,None,hondzik@skaut.cz,mrkosjan@gmail.com,None,None,500,0,"(""Matka: mail"" <mrkosjan@gmail.com>, ""Otec: ma...",()
3,"Unapid prispecky, unpaid STS",Albatrosové,10126,None,None,"mrkosjan@gmail.com, hondzik@skaut.cz",None,None,700,0,"(""Matka: mail"" <mrkosjan@gmail.com>, ""Matka: m...",()
8,Socháň Gabriel,Rackové,20227,hondzik@skaut.cz,None,None,None,None,700,0,"(""E-mail (hlavní)"" <hondzik@skaut.cz>,)",()


In [14]:
with open(f'./test.eml', 'wb') as f:
    f.write(bytes(emails[0]))

In [15]:
mailer = Mailer()
sent = []

[INFO] file_cache is only supported with oauth2client<4.0.0


In [16]:
for email in emails[:]:
    sent_msg = mailer.send_message(email)
    sent.append((email, sent_msg))

[INFO] Message Id: 1864a7272469fa79 sent.
[INFO] Message Id: 1864a72770e82049 sent.
[INFO] Message Id: 1864a72797e4041c sent.
[INFO] Message Id: 1864a727a533d615 sent.
